# atCoderを解いてみる (Part 2)

<p style="text-align: right;">2018.05.08 田辺</p>


## 例題2

Part1で一通りの説明はできたが，入力にはいろいろなパターンがあるので，もう少し説明を加える．

次の問題を考えよう．

> N組の夫婦がパーティに参加した．夫の身長は $a_i$cm,
> 妻の身長は $b_i$cm である ($1 \leq i \leq N$)．
> 妻の身長が夫の身長よりも高い夫婦が何組いたか答えよ．

以下の制約がついているものとする．

> * $1 \leq N \leq 100$
> * $100 \leq a_i \leq 250$
> * $100 \leq b_i \leq 250$
> * 入力中の全ての値は整数である．

入力の形式

> $N$ <br/>
> $a_1$ $b_1$<br/>
> $a_2$ $b_2$<br/>
> ...<br/>
> $a_N$ $b_N$<br/>

入力例1

    4
    186 151
    168 169
    172 161
    177 159

出力例1

    1


## solve

まずsolveから考える．要するに $a_i < b_i$ であるような i の個数を数えれば良いのであるから，主要部は次のようになると
考えられる:

    count = 0
    for i in range(n):
        if a[i] < b[i]:
            count += 1
    return count

関数 solve でこのようなコードが書けるためには，以下のような前提が必要であることがわかる:

* 組数を表す整数nが使えること
* 夫の身長を表す整数のリストaが使えること
* 妻の身長を表す整数のリストbが使えること

つまり，solveは n, a, b を引数として受け取る必要がある:

In [ ]:
def solve(n, a, b):
    count = 0
    for i in range(n):
        if a[i] < b[i]:
            count += 1
    return count

テストしてみよう．

In [ ]:
print(solve(4, [186, 168, 172, 177], [151, 169, 161, 159]))
print(solve(3, [190, 200, 210], [160, 160, 170]))
print(solve(2, [150, 190], [155, 191]))

良いようである．

## readQuestion

まず，なにはともあれsysモジュールをインポートしておく．

In [ ]:
import sys

上の考察から，readQuestion は，n, a, b を設定・作成しなければならないことがわかる．

例題1と異なり，入力が複数行にわたっている．
したがって，`sys.stdin.readlin()` を，行数の回数だけ実行する必要がある．

「行数の回数」といっても，行数はどうしたらわかるか?
それは，最初の行を読んでみればわかる．したがって，以下のようにコードを書けば良い．

* まず，最初の行を読んでみる．
* そこに書かれていた数の回数だけ，行を読み込む．

だいたいこんなコードである:

In [ ]:
def readQuestionA():
    n = sys.stdin.readline()
    for i in range(n):
        line = sys.stdin.readline()
        # さらに処理

残念ながら，このままではうまく動作しない．実行してみるとわかる:

In [ ]:
readQuestionA()

エラーメッセージを直訳すると，「'str'オブジェクトは，整数として解釈することはできない」．
3行目の range(n) で，n はrangeの引数だから整数であることが期待されているのに，
実際には'str'オブジェクト (つまり文字列) だ，と文句を言っているのである．

そこで，文字列を整数に変換する関数 int を用いる．
n が文字列であっても，int(n) とすると，整数が得られる．

In [ ]:
def readQuestionB():
    sn = sys.stdin.readline()
    n = int(sn)
    for i in range(n):
        line = sys.stdin.readline()
        # さらに処理
        print(line.rstrip())   # とりあえず出力してみることにした．

In [ ]:
readQuestionB()

またエラーになってしまった．
今度のエラーメッセージは，「十進整数として，不正なリテラルである '' が与えられた」である．
int という関数は，整数以外のものを整数に変換する関数であるが，何を渡しても良いわけではなく，
十進整数に見えるようなものを渡さなくてはならない．たとえば:

In [ ]:
int('3152')

In [ ]:
int('-413')

などはOKだが，

In [ ]:
int('detarame')

In [ ]:
int('でたらめ')

などは整数にしようがないので，撥ねられる，というわけである．

話を戻して，上の readQuestionB() の実行では，(Jupyter Notebook が標準入力を扱えないために) snに空文字列が設定されてしまい，
そのため，それを十進整数に見ることができずにエラーになったわけである．

実際，以下のコード:

    def readQuestionB():
        sn = sys.stdin.readline()
        n = int(sn)
        for i in range(n):
            line = sys.stdin.readline()
            # さらに処理
            print(line.rstrip())   # とりあえず出力してみることにした．

    readQuestionB()

を，sample.py として保存し，さらに入力例1を sampleData.txt として保存して，
端末で

    python sample.py < sampleData.txt
    
とすれば，エラー無く実行が行われる．

## StringIO

これで良いのだが，readQuestion 関数のテストをするために，いちいちファイルに書き出して端末で
実行しなければならないのは面倒である．
StringIO というものを使うと，もう少し簡単にテストをすることができる．
準備として，ioモジュールをインポートしておく必要がある．

In [ ]:
import io

以下のように使う．まず，標準入力から読み込ませたいデータを，適当な変数に代入する．

In [ ]:
indata = '''4
186 151
168 169
172 161
177 159
'''

三重引用符 `'''` で囲むことで，改行文字も含めることができるようになる．

文字列の先頭は「4」という文字にしたいので，三重引用符の直後に4を置いている．一方，最後の159の直後には改行文字を置きたいので，
最後の三重引用符とその直前の159の間に改行が入っている．

この文字列を io.StringIO() に渡して，sys.stdin に代入することで，標準入力から，この文字列が入力されるように見せかけることができる:

In [ ]:
sys.stdin = io.StringIO(indata)

実行してみよう．

In [ ]:
readQuestionB()

うまくいった．なお，もう一度実行するとエラーになる．

In [ ]:
readQuestionB()

これは，入力という性質上，一度読んでしまうとなくなってしまうからである．
もう一度標準入力の設定をしてから実行すれば大丈夫:

In [ ]:
sys.stdin = io.StringIO(indata)
readQuestionB()

## readQuestion

話を元に戻して，readQuestion 関数の定義を続ける．

nを読んだあとのループ中では，`sys.stdin.readlin()` によって，"186 151\n" のような文字列が返される．
(最後の「\n」は改行文字を表している．) 
これを，2つの整数 186 と 151 に分割しなければならない．

### split

このために用いられるのが，split というメソッドである．文字列 x に対し，x.split(' ') とすると，xを空白で分割してくれる．

In [ ]:
"abc defghi jk".split(' ')

2個以上空白が続くと面倒なことになるが，atCoderではそのようなことはないので，気にしない．

In [ ]:
"abc   defghi      jk".split(' ')

今の問題の場合，各行に2つのデータが書かれていることはわかっている．
このような場合には，受け側にカンマで区切った2つの変数を置いておくことで対応できる:

In [ ]:
v1, v2 = 'hello world'.split(' ')
print('v1 = ', v1)
print('v2 = ', v2)

なお，数が合わないとエラーになる．

In [ ]:
v1, v2, v3 = 'I am a Japanese'.split(' ')

In [ ]:
v1, v2, v3, v4, v5 = 'I am a Japanese'.split(' ')

分割して得られた2つの文字列は，まず　int() を用いて整数にしてから，リスト a, b に格納していく．
ループに入る前に a, b を空リストにしておいて，ループ中で，値を追加していく．
リストに値を追加するためには，メソッド `append` を用いる:

In [ ]:
def readQuestion():
    line = sys.stdin.readline()
    n = int(line)
    a = []
    b = []
    for i in range(n):
        line = sys.stdin.readline()
        sline = line.rstrip()
        s1, s2 = sline.split(' ')
        v1 = int(s1)
        v2 = int(s2)
        a.append(v1)
        b.append(v2)
    return (n, a, b)

最後に返す値は n, a, b の3つである．
python では，カンマで区切って複数の値を1つにまとめることができる．
これを組 (tuple) と呼ぶ．

本質的ではないが，まとめられるところはまとめてしまっても良い．
(無理にまとめなくても良い)

In [ ]:
def readQuestion():
    n = int(sys.stdin.readline())
    a = []
    b = []
    for i in range(n):
        s1, s2 = sys.stdin.readline().rstrip().split(' ')
        a.append(int(s1))
        b.append(int(s2))
    return (n, a, b)

テストしてみよう．

In [ ]:
sys.stdin = io.StringIO(indata)
readQuestion()

良さそうであるが，もう2つか3つはテストしてみた方が良い．

## 統合テスト

最後は StringIO を用いずに，端末上でテストしなければならない．
下のコードをファイルに格納して，テストする．

In [ ]:
import sys

def solve(n, a, b):
    count = 0
    for i in range(n):
        if a[i] < b[i]:
            count += 1
    return count

def readQuestion():
    line = sys.stdin.readline()
    n = int(line)
    a = []
    b = []
    for i in range(n):
        line = sys.stdin.readline()
        sline = line.rstrip()
        s1, s2 = sline.split(' ')
        v1 = int(s1)
        v2 = int(s2)
        a.append(v1)
        b.append(v2)
    return (n, a, b)

def main():
    (n, a, b) = readQuestion()
    answer = solve(n, a, b)
    print(answer)
    
if __name__ == '__main__':
    main()